In [1]:
from dotenv import load_dotenv

In [3]:
load_dotenv("openai.env")

True

In [5]:
from openai import OpenAI
import os

api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

messages = [
    {"role": "user", "content": "介绍下你自己"}
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

print(response.choices[0].message.content)

你好！我是一个人工智能助手，旨在为你提供信息和帮助。我可以回答各种问题，提供建议，帮助你解决问题，或者仅仅是聊天。我具备广泛的知识，涉及科技、历史、文化、语言等多个领域。如果你有什么想知道的或者需要帮助的，随时可以问我！


In [7]:
! pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [ ]:
from langchain.chains import LLMChain,StuffDocumentsChain
from langchain.document_transformers import (
    LongContextReorder
)
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import  Chroma

In [ ]:
embedings = HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")

text = [
    "篮球是一项伟大的运动。",
    "带我飞往月球是我最喜欢的歌曲之一。",
    "凯尔特人队是我最喜欢的球队。",
    "这是一篇关于波士顿凯尔特人的文件。",
    "我非常喜欢去看电影。",
    "波士顿凯尔特人队以20分的优势赢得了比赛。",
    "这只是一段随机的文字。",
    "《艾尔登之环》是过去15年最好的游戏之一。",
    "L.科内特是凯尔特人队最好的球员之一。",
    "拉里.伯德是一位标志性的NBA球员。"
]

retrieval = Chroma.from_texts(text,embedings).as_retriever(
    search_kwargs={"k": 10}
)
query = "关于我的喜好都知道什么?"

docs = retrieval.get_relevant_documents(query)
docs


In [ ]:
reordering = LongContextReorder()
reo_docs = reordering.transform_documents(docs)

reo_docs

In [ ]:
from dotenv import load_dotenv
load_dotenv("openai.env")
import os
api_key = os.environ.get("OPENAI_API_KEY")
api_base = os.environ.get("OPENAI_API_BASE")

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(
    api_key=api_key,
    api_base=api_base,
    model="gpt-3.5-turbo-instruct",
    temperature=0
)

document_prompt = PromptTemplate(
    input_variables=["page_content"],template="{page_content}"
)

stuff_prompt_override ="""Given this text extracts:
----------------------------------------
{context}
----------------------------------------
Please answer the following questions:
{query}
"""

prompt = PromptTemplate(
    template=stuff_prompt_override,
    input_variables=["context","query"]
)

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt
)

WorkChain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name="context"
)

WorkChain.run(
    input_documents=reo_docs,
    query="我最喜欢做什么事情？"
)
